# Dallas Restaurant Analysis

### Researcher(s):  

### Date:  February 1, 2020

### Source(s):
- [Dallas Open Data](https://www.dallasopendata.com)
- [Google](https://www.googlepalcesapi.com)
- [Yelp](https://www.yelpapi.com)

## Summary
#### The purpose of this study...
#### Analysis details (python code) is intentionally viewable.
#### Analysis will provide:
* TBD
* TBD
* TBD

## Analysis Results Overview
#### Initial Analysis ...:
  - 
  - 
  - 

#### Differences ....

*See analysis for details.*

### Possible next steps...
#### ....

## Import and clean Dallas Open Data

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import csv
import gmaps

# Google API Key
from config import gkey

# Configure gmaps with API key
gmaps.configure(api_key=gkey)

%matplotlib inline

In [2]:
# Load in raw Dallas Open Data
raw_dallas_open_data = 'resources/Restaurant_and_Food_Establishment_Inspections__October_2016_to_Present_.csv'

# Read and display the CSV with Pandas
raw_dallas_open_data_pd = pd.read_csv(raw_dallas_open_data, low_memory=False)
#dallas_open_data_pd.head()

In [3]:
# Clean and extract specific columns for 

restaurant_name = raw_dallas_open_data_pd['Restaurant Name'].str.extract('([a-zA-Z0-9&+."\'""\-" ]+)', expand=False).str.strip()
inspection_type = raw_dallas_open_data_pd['Inspection Type']
inspection_date = raw_dallas_open_data_pd['Inspection Date']#.astype('datetime64[ns]')
inspection_score = raw_dallas_open_data_pd['Inspection Score']
street_number = raw_dallas_open_data_pd['Street Number'].astype('str')
street_name = raw_dallas_open_data_pd['Street Name']
street_unit = raw_dallas_open_data_pd['Street Unit'].fillna('').astype('str')

# Return first 5 integers for zip  .str.replace('f', repr)
#zip_code = raw_dallas_open_data_pd['Zip Code'].str.replace('-', '1').str.slice(stop=5).astype(int)
zip_code = raw_dallas_open_data_pd['Zip Code'].str.extract('([0-9]+)', expand=False).str.strip().str.slice(stop=5).astype(int)
street_address = raw_dallas_open_data_pd['Street Address']

# Use the inspection date's month and year for inspection_month for consistency
inspection_month = raw_dallas_open_data_pd['Inspection Date'].astype('datetime64[ns]').dt.strftime('%m/1/%y')

# Dallas Open Data Given Fiscal Year Report Reported
inspection_year = raw_dallas_open_data_pd['Inspection Year']

# Check record count.  Should be 42,070
len(restaurant_name)

42070

In [4]:
#Save data tp dataframe
clean_dallas_open_data_pd = pd.DataFrame({
    'restaurant_name':restaurant_name,
    'inspection_type':inspection_type,
    'inspection_date':inspection_date,
    'inspection_score':inspection_score,
    'street_number':street_number,
    'street_name':street_name,
    'street_unit':street_unit,
    'zip_code':zip_code,
    'street_address':street_address,
    'inspection_month':inspection_month,
    'inspection_year':inspection_year
})
# Confirm expected output & record count of 42,070
#clean_dallas_open_data_pd.info
len(clean_dallas_open_data_pd)

42070

In [5]:
# Keep unique record by street address, Reformat column name for display, & save to dataframe
unique_dallas_open_data_pd = pd.DataFrame({
    'Restaurant Name':restaurant_name,
    'Inspection Type':inspection_type,
    'Inspection Date':inspection_date,
    'Inspection Score':inspection_score,
    'Street Number':street_number,
    'Street Name':street_name,
    'Street Unit':street_unit,
    'Zip':zip_code,
    'Street Address':street_address,
    'Inspection Month':inspection_month,
    'Inspection Year':inspection_year
}).drop_duplicates(subset=['Street Address'])#.reset_index(drop=True, inplace=True)

# Display dataframe (OPTIONAL)
#unique_dallas_open_data_pd

In [6]:
# Identify businesses not of interest
# Consider code refactor
unwanted_restaurants = unique_dallas_open_data_pd[
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-11') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7 - ELEVEN') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7- ELEVEN') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7- ELEVEN CONVENIENCE STORE') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-11 CONVENIENCE STORE') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-11CONVENIENCE STORE') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-ELEVEN') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-ELEVEN CONVENIENCE') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-ELEVEN CONVENIENCE STORE') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-ELEVEN CONVENIENE STORE') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-ELEVEN STORE') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7 ELEVEN') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-7-ELEVEN') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '99 CENTS ONLY') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '99 CENTS ONLY STORE')
].index

# Drop unwanted businesses to reduce api calls to google/yelp
unique_dallas_open_data_pd.drop(unwanted_restaurants, inplace=True)

# Reset datafram index
unique_dallas_open_data_pd.reset_index(drop=True, inplace=True)

In [7]:
# Output File (CSV) for validation (OPTIONAL)
#unique_dallas_open_data_output = "./output_data/unique_bus_dal_open_data_2019.csv"

# write dataframe to the csv
#unique_dallas_open_data_pd.to_csv(unique_dallas_open_data_output, sep=',', encoding='utf-8', index=False)

In [8]:
g_lookup_pd_rn = unique_dallas_open_data_pd
g_lookup_pd_rn["Lat"] = ""
g_lookup_pd_rn["Long"] = ""
g_lookup_pd_rn["Type"] = ""
g_lookup_pd_rn["Rating"] = ""
g_lookup_pd_rn["Rating Count"] = ""

g_lookup_pd_rn

,Restaurant Name,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,Street Address,Inspection Month,Inspection Year,Lat,Long,Type,Rating,Rating Count
0,FRESHII,Routine,10/31/2018,96,2414,VICTORY PARK,,75219,2414 VICTORY PARK LN,10/1/18,FY2019,,,,,
1,MICKLE CHICKEN,Routine,10/30/2019,100,3203,CAMP WISDOM,,75237,3203 W CAMP WISDOM RD,10/1/19,FY2020,,,,,
2,WORLD TRADE CENTER MARKET,Routine,11/03/2016,100,2050,STEMMONS,,75207,2050 N STEMMONS FRWY,11/1/16,FY2017,,,,,
3,DUNKIN DONUTS,Routine,10/30/2019,99,8008,HERB KELLEHER,C2174,75235,8008 HERB KELLEHER WAY STE# C2174,10/1/19,FY2020,,,,,
4,CANVAS HOTEL - 6TH FLOOR,Routine,06/11/2018,100,1325,LAMAR,,75215,1325 S LAMAR ST,06/1/18,FY2018,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6633,THE TIPSY CUPCAKE,Routine,02/27/2019,91,2400,HENDERSON,#B,75219,2400 N HENDERSON #B,02/1/19,FY2019,,,,,
6634,FOOD PLUS,Routine,02/17/2017,94,6214,SAMUELL,,75228,6214 SAMUELL BLVD,02/1/17,FY2017,,,,,
6635,TOP POT DOUGHNUTS,Routine,04/04/2017,90,2320,HOUSTON,#100,75202,2320 N HOUSTON #100,04/1/17,FY2017,,,,,
6636,LA MICHOACANA PREMIER ICE CREAM,Routine,11/12/2018,91,10325,LAKE JUNE,#550,75182,10325 LAKE JUNE RD #550,11/1/18,FY2019,,,,,


In [11]:
# Create a copy of re-organized Data
g_lookup_pd_filter = g_lookup_pd_rn.copy()

# Filter by Inspection Score
#g_lookup_pd_filter = g_lookup_pd_filter[g_lookup_pd_filter['Inspection Score'] == 83]

# Filter by Zip
g_lookup_pd_filter = g_lookup_pd_filter[g_lookup_pd_filter['Zip'] == 75219]

g_lookup_pd_filter = g_lookup_pd_filter.reset_index()

g_lookup_pd_filter = g_lookup_pd_filter.head(50)

g_lookup_pd_filter

,index,Restaurant Name,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,Street Address,Inspection Month,Inspection Year,Lat,Long,Type,Rating,Rating Count
0,0,FRESHII,Routine,10/31/2018,96,2414,VICTORY PARK,,75219,2414 VICTORY PARK LN,10/1/18,FY2019,,,,,
1,134,KROGER,Routine,04/27/2017,100,4142,CEDAR SPRINGS,,75219,4142 CEDAR SPRINGS RD,04/1/17,FY2017,,,,,
2,159,ZOE'S KITCHEN,Routine,05/11/2017,98,3878,OAK LAWN,#314,75219,3878 OAK LAWN #314,05/1/17,FY2017,,,,,
3,171,MANSION MAIN BAR,Routine,10/10/2017,99,2821,TURTLE CREEK,,75219,2821 TURTLE CREEK BLVD,10/1/17,FY2018,,,,,
4,190,PERSONALIZED LEARNING PREPARATORY AT SAM HOUSTON,Routine,05/23/2019,96,2827,THROCKMORTON,,75219,2827 THROCKMORTON ST,05/1/19,FY2019,,,,,
5,217,MEDRANO ELEMENTARY SCHOOL,Routine,11/04/2019,100,2221,LUCAS,,75219,2221 LUCAS DR,11/1/19,FY2020,,,,,
6,241,SILVERADO SENIOR LIVING-TURTLE CREEK,Routine,06/13/2019,97,3611,DICKASON,,75219,3611 DICKASON AVE,06/1/19,FY2019,,,,,
7,372,SAL'S PIZZA,Routine,06/12/2019,90,2525,WYCLIFF,#116,75219,2525 WYCLIFF AVE #116,06/1/19,FY2019,,,,,
8,395,SCOTTISH RITE HOSPITAL SERVERY,Routine,05/30/2019,96,2222,WELBURN,,75219,2222 WELBURN ST,05/1/19,FY2019,,,,,
9,414,MURPHYS DELI,Routine,06/13/2019,90,2323,VICTORY,#150,75219,2323 VICTORY AVE #150,06/1/19,FY2019,,,,,


In [12]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

# Loop through the g_lookup_pd_org and run a lat/long search for each city
for index, row in g_lookup_pd_filter.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/geocode/json"

        street = row['Street Address']
        zip_code = row['Zip']
        restaurantname = row['Restaurant Name']
        restaurantname = restaurantname.title()

        # update address key value
        params['address'] = f"{street},{zip_code}"

        # make request
        cities_lat_lng = requests.get(base_url, params=params)

        # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
        
        # convert to json
        cities_lat_lng = cities_lat_lng.json()
        
        g_lookup_pd_filter.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        #lat = g_lookup_pd_filter.loc[index, "Lat"]
        
        g_lookup_pd_filter.loc[index, "Long"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
        #long = g_lookup_pd_filter.loc[index, "Long"]
        
        print(f'''[OK!] {restaurantname} found!''')            

    except:
        print(f'''[ERROR!] {restaurantname} missing field/result.''')

        
        pass

[OK!] Freshii found!
[OK!] Kroger found!
[OK!] Zoe'S Kitchen found!
[OK!] Mansion Main Bar found!
[OK!] Personalized Learning Preparatory At Sam Houston found!
[OK!] Medrano Elementary School found!
[OK!] Silverado Senior Living-Turtle Creek found!
[OK!] Sal'S Pizza found!
[OK!] Scottish Rite Hospital Servery found!
[OK!] Murphys Deli found!
[OK!] Billy Can Can found!
[OK!] Liquid Zoo Sports & Cigar Bar found!
[OK!] Accent Food Services found!
[OK!] Kfc found!
[OK!] Chocolate Secrets found!
[OK!] Buzzbrews found!
[OK!] Imoto-1St Floor found!
[OK!] Roots Pressed Juices found!
[OK!] Jack In The Box found!
[OK!] Jr'S Bar found!
[OK!] Hero By Hg Supply found!
[OK!] Green Papaya found!
[OK!] East Hampton Sandwich Co found!
[OK!] Chef House Restaurant found!
[OK!] Tacos Y Mas found!
[OK!] Belmont Village Turtle Creek Tenant found!
[OK!] Starbucks found!
[OK!] Holy Trinity School found!
[OK!] Wingstop found!
[OK!] Dickey'S Barbecue Pit found!
[OK!] Dibs found!
[OK!] Eatzi'S Market Chef Cafe f

In [13]:
# params dictionary to update each iteration
# radius of 50 has been giving me a 90% match for Type, Rating, and Rating Count
params = {
    "radius": 50,
    "types": "restaurant",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in g_lookup_pd_filter.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Long"]
    restaurantname = row["Restaurant Name"]
    restaurantname = restaurantname.title()

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        g_lookup_pd_filter.loc[index, "Type"] = name_address["results"][0]["types"][0].title()
        g_lookup_pd_filter.loc[index, "Rating"] = name_address["results"][0]["rating"]
        g_lookup_pd_filter.loc[index, "Rating Count"] = name_address["results"][0]["user_ratings_total"]
        
        print(f'''[OK!] {restaurantname} found!''')
        
    
    except (KeyError, IndexError):
        print(f'''[ERROR!] {restaurantname} missing field/result.''')

[OK!] Freshii found!
[ERROR!] Kroger missing field/result.
[OK!] Zoe'S Kitchen found!
[OK!] Mansion Main Bar found!
[ERROR!] Personalized Learning Preparatory At Sam Houston missing field/result.
[ERROR!] Medrano Elementary School missing field/result.
[ERROR!] Silverado Senior Living-Turtle Creek missing field/result.
[OK!] Sal'S Pizza found!
[ERROR!] Scottish Rite Hospital Servery missing field/result.
[OK!] Murphys Deli found!
[OK!] Billy Can Can found!
[OK!] Liquid Zoo Sports & Cigar Bar found!
[OK!] Accent Food Services found!
[OK!] Kfc found!
[OK!] Chocolate Secrets found!
[OK!] Buzzbrews found!
[OK!] Imoto-1St Floor found!
[OK!] Roots Pressed Juices found!
[OK!] Jack In The Box found!
[OK!] Jr'S Bar found!
[OK!] Hero By Hg Supply found!
[OK!] Green Papaya found!
[ERROR!] East Hampton Sandwich Co missing field/result.
[OK!] Chef House Restaurant found!
[OK!] Tacos Y Mas found!
[ERROR!] Belmont Village Turtle Creek Tenant missing field/result.
[OK!] Starbucks found!
[ERROR!] Holy 

In [14]:
g_lookup_pd_filter

,index,Restaurant Name,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,Street Address,Inspection Month,Inspection Year,Lat,Long,Type,Rating,Rating Count
0,0,FRESHII,Routine,10/31/2018,96,2414,VICTORY PARK,,75219,2414 VICTORY PARK LN,10/1/18,FY2019,32.7879,-96.8092,Restaurant,4.1,100
1,134,KROGER,Routine,04/27/2017,100,4142,CEDAR SPRINGS,,75219,4142 CEDAR SPRINGS RD,04/1/17,FY2017,32.8134,-96.8121,,,
2,159,ZOE'S KITCHEN,Routine,05/11/2017,98,3878,OAK LAWN,#314,75219,3878 OAK LAWN #314,05/1/17,FY2017,32.8155,-96.8007,Restaurant,4.2,334
3,171,MANSION MAIN BAR,Routine,10/10/2017,99,2821,TURTLE CREEK,,75219,2821 TURTLE CREEK BLVD,10/1/17,FY2018,32.8041,-96.8073,Bar,4.6,433
4,190,PERSONALIZED LEARNING PREPARATORY AT SAM HOUSTON,Routine,05/23/2019,96,2827,THROCKMORTON,,75219,2827 THROCKMORTON ST,05/1/19,FY2019,32.8102,-96.8131,,,
5,217,MEDRANO ELEMENTARY SCHOOL,Routine,11/04/2019,100,2221,LUCAS,,75219,2221 LUCAS DR,11/1/19,FY2020,32.8088,-96.8266,,,
6,241,SILVERADO SENIOR LIVING-TURTLE CREEK,Routine,06/13/2019,97,3611,DICKASON,,75219,3611 DICKASON AVE,06/1/19,FY2019,32.807,-96.8083,,,
7,372,SAL'S PIZZA,Routine,06/12/2019,90,2525,WYCLIFF,#116,75219,2525 WYCLIFF AVE #116,06/1/19,FY2019,32.811,-96.8191,Restaurant,4,232
8,395,SCOTTISH RITE HOSPITAL SERVERY,Routine,05/30/2019,96,2222,WELBURN,,75219,2222 WELBURN ST,05/1/19,FY2019,32.8023,-96.8139,,,
9,414,MURPHYS DELI,Routine,06/13/2019,90,2323,VICTORY,#150,75219,2323 VICTORY AVE #150,06/1/19,FY2019,32.7856,-96.8106,Restaurant,1,1


In [15]:
# Save as CSV
g_lookup_pd_filter.to_csv("./output_data/Google Lat Long Results.csv")